# 클라우드에서 데이터 과학: "Azure ML SDK" 방식

## 소개

이 노트북에서는 Azure ML SDK를 사용하여 모델을 학습, 배포 및 소비하는 방법을 배워보겠습니다.

사전 준비 사항:
1. Azure ML 워크스페이스를 생성했습니다.
2. [Heart Failure 데이터셋](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data)을 Azure ML에 로드했습니다.
3. 이 노트북을 Azure ML Studio에 업로드했습니다.

다음 단계는 다음과 같습니다:

1. 기존 워크스페이스에서 실험을 생성합니다.
2. 컴퓨팅 클러스터를 생성합니다.
3. 데이터셋을 로드합니다.
4. AutoMLConfig를 사용하여 AutoML을 구성합니다.
5. AutoML 실험을 실행합니다.
6. 결과를 탐색하고 최적의 모델을 확인합니다.
7. 최적의 모델을 등록합니다.
8. 최적의 모델을 배포합니다.
9. 엔드포인트를 소비합니다.

## Azure Machine Learning SDK 관련 임포트


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## 작업 공간 초기화  
저장된 구성에서 작업 공간 객체를 초기화합니다. 구성 파일이 .\config.json에 있는지 확인하세요.  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Azure ML 실험 생성

방금 초기화한 작업 영역에서 'aml-experiment'라는 이름의 실험을 만들어봅시다.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## 컴퓨트 클러스터 생성
AutoML 실행을 위해 [컴퓨트 대상](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target)을 생성해야 합니다.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## 데이터
데이터셋을 Azure ML에 업로드했으며, 키가 데이터셋과 동일한 이름인지 확인하세요.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML 구성


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML 실행


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## 최적의 모델 배포하기

다음 코드를 실행하여 최적의 모델을 배포하세요. 배포 상태는 Azure ML 포털에서 확인할 수 있습니다. 이 단계는 몇 분 정도 소요될 수 있습니다.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## 엔드포인트 사용하기
다음 입력 샘플에 입력값을 추가할 수 있습니다.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
